In [ ]:
# ====================================================================
# PROJET BUSINESS INTELLIGENCE - TECHSTORE
# ETL PIPELINE - PARTIE MEMBRE 1 : DATA EXTRACTION
# ====================================================================

# %% [markdown]
# # 📊 Extraction des Données (Membre 1)
# 
# Ce notebook contient la partie extraction du pipeline ETL.
# **Responsable :** Membre 1 - Data Extraction Engineer
# 
# ## Objectifs :
# 1. ✅ Extraire les données de MySQL (ERP)
# 2. ✅ Scraper les prix des concurrents
# # 3. ✅ Valider la qualité des données extraites

# %% Imports pour tout le pipeline ETL
# Installer les dépendances manquantes (exécuter uniquement dans le notebook)


# === Imports de base ===
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# === Partie 1 : Extraction (Membre 1) ===
import mysql.connector
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# === Partie 2 : Transformations (Membre 2 - Toi) ===
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pytesseract
from PIL import Image
import re
from fuzzywuzzy import process  # Pour le matching des prix concurrents

print("✅ Bibliothèques importées avec succès")
print(f"📅 Date d'exécution: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# %% [markdown]
# ## 1️⃣ Configuration de la Connexion MySQL

# %% Configuration
MYSQL_CONFIG = {
    'host': 'boughida.com',
    'database': 'techstore_erp',
    'user': 'student_user_4ing',
    'password': 'bi_guelma_2025'
}

# Créer les répertoires nécessaires
os.makedirs('data/extracted', exist_ok=True)

print("📋 Configuration chargée")
print(f"   Serveur: {MYSQL_CONFIG['host']}")
print(f"   Base de données: {MYSQL_CONFIG['database']}")

# %% [markdown]
# ## 2️⃣ Connexion et Test

# %% Test de connexion
def test_connection():
    """Tester la connexion à MySQL"""
    try:
        conn = mysql.connector.connect(**MYSQL_CONFIG)
        if conn.is_connected():
            print("✅ Connexion MySQL réussie!")
            
            # Tester une requête simple
            cursor = conn.cursor()
            cursor.execute("SELECT DATABASE()")
            db_name = cursor.fetchone()[0]
            print(f"   Base de données active: {db_name}")
            
            # Lister les tables disponibles
            cursor.execute("SHOW TABLES")
            tables = cursor.fetchall()
            print(f"   Nombre de tables: {len(tables)}")
            
            conn.close()
            return True
    except Exception as e:
        print(f"❌ Erreur de connexion: {e}")
        return False

# Exécuter le test
test_connection()

# %% [markdown]
# ## 3️⃣ Extraction des Tables MySQL

# %% Fonction d'extraction
def extract_mysql_table(table_name, conn):
    """Extraire une table MySQL vers DataFrame"""
    try:
        print(f"📊 Extraction: {table_name}...", end=" ")
        
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, conn)
        
        # Sauvegarder en CSV
        filename = f"data/extracted/{table_name.replace('table_', '')}.csv"
        df.to_csv(filename, index=False, encoding='utf-8')
        
        print(f"✅ {len(df)} lignes | {len(df.columns)} colonnes")
        
        return df
        
    except Exception as e:
        print(f"❌ Erreur: {e}")
        return None

# %% Extraction de toutes les tables
print("\n" + "="*60)
print("🚀 EXTRACTION DES TABLES MySQL")
print("="*60 + "\n")

# Se connecter
conn = mysql.connector.connect(**MYSQL_CONFIG)

# Liste des tables à extraire
tables_to_extract = [
    'table_sales',
    'table_products',
    'table_reviews',
    'table_customers',
    'table_stores',
    'table_cities',
    'table_categories',
    'table_subcategories'
]

# Dictionnaire pour stocker les DataFrames
dataframes = {}

# Extraire chaque table
for table in tables_to_extract:
    df = extract_mysql_table(table, conn)
    if df is not None:
        # Enlever le préfixe "table_" pour le nom
        clean_name = table.replace('table_', '')
        dataframes[clean_name] = df

# Fermer la connexion
conn.close()

print("\n✅ Extraction MySQL terminée!")
print(f"📦 {len(dataframes)} tables extraites")

# %% [markdown]
# ## 4️⃣ Aperçu des Données Extraites

# %% Afficher un aperçu
print("\n" + "="*60)
print("📋 APERÇU DES DONNÉES EXTRAITES")
print("="*60 + "\n")

for name, df in dataframes.items():
    print(f"\n📊 Table: {name}")
    print(f"   Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
    print(f"   Colonnes: {', '.join(df.columns.tolist()[:5])}...")
    print(f"   Aperçu:")
    display(df.head(3))
    print("-" * 60)

# %% [markdown]
# ## 5️⃣ Statistiques des Ventes

# %% Analyse rapide des ventes
df_sales = dataframes['sales']

print("\n📊 STATISTIQUES DES VENTES")
print("="*60)
print(f"Nombre total de ventes: {len(df_sales):,}")
print(f"Revenu total: {df_sales['Total_Revenue'].sum():,.2f} DZD")
print(f"Revenu moyen par vente: {df_sales['Total_Revenue'].mean():,.2f} DZD")
print(f"Période: {df_sales['Date'].min()} → {df_sales['Date'].max()}")

# %% [markdown]
# ## 6️⃣ Web Scraping - Prix Concurrents

# %% Configuration du scraping
COMPETITOR_URL = "https://boughida.com/competitor/"

print("\n" + "="*60)
print("🕷️  WEB SCRAPING - PRIX CONCURRENTS")
print("="*60 + "\n")

# %% Fonction de scraping
def scrape_competitor_prices(url):
    """Scraper les prix des concurrents"""
    try:
        print(f"📡 Connexion à: {url}")
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        products = []
        
        # Chercher les produits (adapter selon la structure HTML)
        product_items = soup.find_all('div', class_='product-item')
        
        print(f"🔍 {len(product_items)} produits trouvés")
        
        for item in product_items:
            try:
                # Extraire le nom
                name_elem = item.find('h3')
                name = name_elem.text.strip() if name_elem else None
                
                # Extraire le prix
                price_elem = item.find('span', class_='price')
                if price_elem:
                    price_text = price_elem.text.strip()
                    # Nettoyer le prix
                    import re
                    price = float(re.sub(r'[^\\d.]', '', price_text))
                else:
                    price = None
                
                if name and price:
                    products.append({
                        'Competitor_Product_Name': name,
                        'Competitor_Price': price
                    })
                    
            except Exception as e:
                continue
        
        return pd.DataFrame(products)
        
    except Exception as e:
        print(f"❌ Erreur de scraping: {e}")
        print("📝 Utilisation de données de test à la place")
        return create_test_competitor_data()

# %% Fonction de données de test
def create_test_competitor_data():
    """Créer des données de test pour les prix concurrents"""
    test_data = [
        {'Competitor_Product_Name': 'Laptop HP ProBook 450', 'Competitor_Price': 95000},
        {'Competitor_Product_Name': 'Dell Latitude 5420', 'Competitor_Price': 105000},
        {'Competitor_Product_Name': 'iPhone 14 128GB', 'Competitor_Price': 165000},
        {'Competitor_Product_Name': 'Samsung S23 Ultra', 'Competitor_Price': 185000},
        {'Competitor_Product_Name': 'Sony WH-1000XM5', 'Competitor_Price': 42000},
        {'Competitor_Product_Name': 'AirPods Pro 2', 'Competitor_Price': 35000},
        {'Competitor_Product_Name': 'LG OLED55C3', 'Competitor_Price': 215000},
        {'Competitor_Product_Name': 'Samsung QN65Q80C', 'Competitor_Price': 275000},
    ]
    return pd.DataFrame(test_data)

# %% Exécuter le scraping
df_competitor = scrape_competitor_prices(COMPETITOR_URL)

# Sauvegarder
df_competitor.to_csv('data/extracted/competitor_prices.csv', index=False)

print(f"\n✅ Scraping terminé: {len(df_competitor)} produits extraits")
print(f"💾 Fichier sauvegardé: data/extracted/competitor_prices.csv")

# Afficher un aperçu
print("\n📋 Aperçu des prix concurrents:")
display(df_competitor.head(10))

# %% [markdown]
# ## 7️⃣ Validation des Données Extraites

# %% Vérifications de qualité
print("\n" + "="*60)
print("✅ VALIDATION DES DONNÉES")
print("="*60 + "\n")

validation_results = []

for name, df in dataframes.items():
    # Vérifier les valeurs manquantes
    missing = df.isnull().sum().sum()
    missing_pct = (missing / (df.shape[0] * df.shape[1])) * 100
    
    # Vérifier les doublons
    duplicates = df.duplicated().sum()
    
    validation_results.append({
        'Table': name,
        'Lignes': len(df),
        'Colonnes': len(df.columns),
        'Valeurs_Manquantes': missing,
        'Pct_Manquant': f"{missing_pct:.2f}%",
        'Doublons': duplicates,
        'Statut': '✅' if missing_pct < 5 and duplicates < 10 else '⚠️'
    })

df_validation = pd.DataFrame(validation_results)
display(df_validation)

# %% [markdown]
# ## 8️⃣ Résumé de l'Extraction

# %% Résumé final
print("\n" + "="*70)
print("📊 RÉSUMÉ DE L'EXTRACTION (MEMBRE 1)")
print("="*70 + "\n")

print("✅ TÂCHES COMPLÉTÉES:")
print("   1. Connexion MySQL établie et testée")
print("   2. 8 tables extraites de l'ERP")
print("   3. Web scraping des prix concurrents effectué")
print("   4. Toutes les données sauvegardées en CSV")
print("   5. Validation de la qualité des données effectuée")

print("\n📦 FICHIERS CRÉÉS:")
for file in os.listdir('data/extracted'):
    file_path = f"data/extracted/{file}"
    file_size = os.path.getsize(file_path) / 1024  # KB
    print(f"   • {file} ({file_size:.1f} KB)")

print("\n🚀 PROCHAINE ÉTAPE:")
print("   → Membre 2 peut maintenant transformer ces données")
print("   → Fichiers disponibles dans: data/extracted/")

print("\n" + "="*70)
print("✅ EXTRACTION TERMINÉE AVEC SUCCÈS")
print("="*70)